In [ ]:
import requests
import json

url = "https://realty-mole-property-api.p.rapidapi.com/zipCodes/07030"

headers = {
	"X-RapidAPI-Key": "0b52c0e7fcmshaed8f0f998397c0p182f41jsn7900be6276c2",
	"X-RapidAPI-Host": "realty-mole-property-api.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)
data = response.json()
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)
print(data)

In [ ]:
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
import lxml
from lxml.html.soupparser import fromstring
#import prettify
import numbers
#import htmltext



req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

with requests.session() as session:
    url = 'https://www.zillow.com/hoboken-nj/rentals/'
    r1 = session.get(url, headers=req_headers)
    #print(r1.content)
    soup = BeautifulSoup(r1.content, 'html.parser')
    
address = soup.find_all('div', class_='zillow-map-control')
price = list(soup.find_all (class_='list-card-price'))
print(price)
print(soup)
#response = requests.get(url)
#print(response.json)


In [ ]:
import requests
import re
import json
import pandas as pd
import csv


city = 'hoboken/'


url1 = 'https://www.zillow.com/homes/for_sale/'+city
url2 = 'https://www.zillow.com/homes/for_sale/'+city+'/2_p/'
url3 = 'https://www.zillow.com/homes/for_sale/'+city+'/3_p/'
url4 = 'https://www.zillow.com/homes/for_sale/'+city+'/4_p/'
url5 = 'https://www.zillow.com/homes/for_sale/'+city+'/5_p/'
url6 = 'https://www.zillow.com/homes/for_sale/'+city+'/6_p/'
url7 = 'https://www.zillow.com/homes/for_sale/'+city+'/7_p/'
url8 = 'https://www.zillow.com/homes/for_sale/'+city+'/8_p/'
url9 = 'https://www.zillow.com/homes/for_sale/'+city+'/9_p/'
url10 = 'https://www.zillow.com/homes/for_sale/'+city+'/10_p/'

req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}


with requests.Session() as s:
    r1 = s.get(url1, headers=req_headers)
    r2 = s.get(url2, headers=req_headers)
    r3 = s.get(url3, headers=req_headers)
    r4 = s.get(url4, headers=req_headers)
    r5 = s.get(url5, headers=req_headers)
    r6 = s.get(url6, headers=req_headers)
    r7 = s.get(url7, headers=req_headers)
    r8 = s.get(url8, headers=req_headers)
    r9 = s.get(url9, headers=req_headers)
    r10 = s.get(url10, headers=req_headers)

    data1 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r1.text).group(1))
    data2 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r2.text).group(1))
    data3 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r3.text).group(1))
    data4 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r4.text).group(1))
    data5 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r5.text).group(1))
    data6 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r6.text).group(1))
    data7 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r7.text).group(1))
    data8 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r8.text).group(1))
    data9 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r9.text).group(1))
    data10 = json.loads(re.search(r'!--(\{"queryState".*?)-->', r10.text).group(1))

data_list = [data1,data2,data3,data4,data5,data6,data7,data8,data9,data10]


df = pd.DataFrame()

def make_frame(frame):
    for i in data_list:
        for item in i['cat1']['searchResults']['listResults']:
            frame = frame.append(item, ignore_index=True)
    return frame

df = make_frame(df)
    
#drop cols
df = df.drop('hdpData', 1) #remove this line to see a whole bunch of other random cols, in dict format

#drop dupes
df = df.drop_duplicates(subset='zpid', keep="last")

#filters
df['zestimate'] = df['zestimate'].fillna(0)
df['best_deal'] = df['unformattedPrice'] - df['zestimate']
df = df.sort_values(by='best_deal',ascending=True)
#df.drop(columns=df.columns[0],axis=1,inplace=True)
#df.reset_index(drop=True)
df = df.iloc[: , 1:]
csv = df.to_csv() 
with open('hoboken_12_22.csv', 'w') as file:
    file.write(csv)

print('shape:', df.shape)
print(df[['id','address','beds','baths','area','price','zestimate','best_deal']].head(100))

In [10]:
import requests

url = "{{HOST}}/client/{{VERSION_CLIENT}}/rentalizer/estimate?access_token={{CLIENT_TOKEN}}&address=3036 Tennyson St Denver co&zipcode=80212&bedrooms=3&bathrooms=2&accommodates=6"

payload={}
headers = {
  'Accept': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

MissingSchema: Invalid URL '{{HOST}}/client/{{VERSION_CLIENT}}/rentalizer/estimate?access_token={{CLIENT_TOKEN}}&address=3036 Tennyson St Denver co&zipcode=80212&bedrooms=3&bathrooms=2&accommodates=6': No scheme supplied. Perhaps you meant http://{{HOST}}/client/{{VERSION_CLIENT}}/rentalizer/estimate?access_token={{CLIENT_TOKEN}}&address=3036 Tennyson St Denver co&zipcode=80212&bedrooms=3&bathrooms=2&accommodates=6?